In [12]:
# Importing packages
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import numpy as np
from ipywidgets import Layout
import seaborn as sns
import math

import time
from scipy import linalg
from scipy import optimize
import sympy as sm

from tkinter import *
import tkinter as tk


# local module for linear algebra
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Straight out copied from Monty P. 

## Importing and cleaning the data 

filename = 'imdb.csv'

## Defining the function to import and clean the data
def gen_df(filename):
    """ Returns a pandas dataframe containing information about movies from the IMDB database 

    Args:
    filename (csv-file): The csv-file have to be the file imdb.csv

    Returns:
    A pandas dataframe

    Notice:
    The function will not work if other files than imdb.csv are used as input """
        
    # Get .csv.file
    data = pd.read_csv(filename, sep=';', encoding='latin-1', escapechar='\\')

    # Read file into pandas dataframe
    df = pd.DataFrame(data)

    # Drop unwanted columns
    for i in range(44,48):
        df.drop(columns=[f'Unnamed: {i}'], inplace=True)
    df.drop(columns=['fn','wordsInTitle','url'], inplace=True)

    # Keep only observations of 'movie'-type
    I = df['type'] == 'video.movie'
    df = df.loc[I]
    df.drop(columns=['type'], inplace=True)

    # Drop movies with missing observations
    df.dropna(inplace=True)

    # During the importing process, the first decimal has become 0 for all movies.
    # Thus, we replace "0"'s with ""
    df['imdbRating'] = df['imdbRating'].astype(str)
    df['imdbRating'].replace(regex=True, inplace=True,to_replace='0',value='')
    df['imdbRating'] = df['imdbRating'].astype(float)

    # Transform duration from seconds to hours
    df['duration'] = df['duration']/60**2

    # Drop years before 1920 and the year 2014 because of few obervations
    I = (df['year']>=1920) & (df['year']<=2013)
    df = df.loc[I]

    # Change the type of 'year' to integer
    df['year'] = df['year'].astype(int)    
    
    # Sort observations and reset index
    df.sort_values('year', inplace=True)
    df.reset_index(inplace=True)

    # Generating a variable that shows the decade (as a string)
    year_list = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010]

    df['decade'] = ''
    for i,start in enumerate(year_list):
        end = start+10
        df.loc[(df['year'] >= start) & (df['year'] < end), 'decade'] = f'{year_list[i]}s'

    return df

df = gen_df(filename)


In [14]:
# Calculate utility (First try)

df["rating_scaled"] = (df["imdbRating"])/10

alpha = 0.5
beta = 0.3
delta = 0.2
alpha1 = 0.5
alpha2 = 0.3
alpha3 = 0.2

def utility(g1="Drama", g2="Action", g3="Romance", decade="2000s"): 
    df["utility"] = 0
    if decade == decade:
        df["utility"] = alpha*(alpha1 * df[g1] + alpha2*df[g2] + alpha3*df[g3]) + beta + delta*df["rating_scaled"]
    else: 
        df["utility"] = alpha*(alpha1 * df[g1] + alpha2*df[g2] + alpha3*df[g3]) + delta*df["rating_scaled"]
utility()

df.sort_values("utility", axis = 0, ascending = False, inplace = True, na_position ='last') 
    
df[["title", "imdbRating", "year", "Drama", "Action", "Romance", "rating_scaled", "decade", "utility"]].head(10)

# Alternate utility function to call user function, and calculate utility based on user type. 
# Also, figure out how to make know which decade comes before and after "decade"

,title,imdbRating,year,Drama,Action,Romance,rating_scaled,decade,utility
8155,Omkara (2006),8.1,2006,1.0,1.0,1.0,0.81,2000s,0.962
7331,Virumandi (2004),7.9,2004,1.0,1.0,1.0,0.79,2000s,0.958
6146,Tiger & Dragon (2000),7.9,2000,1.0,1.0,1.0,0.79,2000s,0.958
9038,Magadheera (2009),7.6,2009,1.0,1.0,1.0,0.76,2000s,0.952
2636,Yaadon Ki Baaraat (1973),7.4,1973,1.0,1.0,1.0,0.74,1970s,0.948
2539,TodesgrÌ_ÌÙe aus Shanghai (1972),7.4,1972,1.0,1.0,1.0,0.74,1970s,0.948
299,The Marines Are Coming (1934),7.3,1934,1.0,1.0,1.0,0.73,1930s,0.946
408,...dann kam der Orkan (1937),7.3,1937,1.0,1.0,1.0,0.73,1930s,0.946
4778,1942: A Love Story (1994),7.3,1994,1.0,1.0,1.0,0.73,1990s,0.946
8073,Der Fluch der goldenen Blume (2006),7.0,2006,1.0,1.0,1.0,0.70,2000s,0.940


In [16]:
# Lists of decades and genres
# decade = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010]
decades = ["1920s", "1930s", "1940s", "1950s", "1960s", "1970s", "1980s", "1990s", "2000s", "2010s"]
genre = list(df)[13:41]

# Defining symbols 
sm.init_printing(use_unicode=True) # for pretty printing

g1 = sm.symbols("G_1") 
g2 = sm.symbols("G_2") 
g3 = sm.symbols("G_3") 
Gi = sm.symbols("G_i")
Dt = sm.symbols("D_t")
Dt1 = sm.symbols("D_t-1")
Dt2 = sm.symbols("D_t+1")
alpha1 = sm.symbols("alpha_1")
alpha2 = sm.symbols("alpha_2")
alpha3 = sm.symbols("alpha_3")
alphai = sm.symbols("alpha_i")
alpha = sm.symbols("alpha")
beta = sm.symbols("beta")
delta = sm.symbols("delta")
R = sm.symbols("R")
T = sm.symbols("T")


# Utility function
test = sm.Eq(1/2 * g1**alpha1 + g2**alpha2 + g3**alpha3,T)

utility_func = sm.Eq(alpha * (alpha1*g1 + alpha2*g2 + alpha3*g3) + beta*Dt + delta*R, T)

sm.init_printing(use_unicode=False)

In [17]:
# Define user function to be called in the function which calculates utility
# Rewrite it to use input functions, such that we interact with the user 

genre1 = input("What is your favorite genre: ")
genre2 = input("What is your second favorite genre: ")
genre3 = input("What is your third favorite genre: ")

def moviefan(g1, g2, g3, decade, first, second, third): 
    """ Defining the Moviefan who needs recommendations
    
    Args: 
        g1: Favorite genre (I.e. "Drama")
        g2: Second favorite genre
        g3: Third favorite genre 
        
        decade: Favorite movie decade (I.e. "2000s")
        
        first: Top parameter (I.e. if genre is most important, choose genre)
        second: Second parameter 
        third: Third parameter 

    The function is used to call in utility function, which calculates a utility for each movie possible. 
    """
    
    first = 1
    second = 2
    third = 3
    





What is your favorite genre:  Drama
What is your second favorite genre:  Romance
What is your third favorite genre:  Action


In [18]:
test = input("Enter a value: ") 
print(f"You typed {test}")

Enter a value:  10


You typed 10


In [32]:
genre = list(df)[13:41]
# genre1 = input("What is your favorite genre: ")

genre1 = widgets.interact(
                genre_test = widgets.Dropdown(
                description="Genres", 
                options=genre,
                value="Drama"))


In [25]:
genre_list = {'Action', 'Adult', 'Adventure', 'Animation', 'Biography',
         'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
         'FilmNoir', 'GameShow', 'History', 'Horror', 'Music', 'Musical',
         'Mystery', 'News', 'RealityTV', 'Romance', 'SciFi', 'Short', 'Sport',
         'TalkShow', 'Thriller', 'War', 'Western'}

root = Tk() # Create tk object
root.title("Favorite genre")

# Add a grid
mainframe = Frame(root)                           # Pass tk object to tkinter frame
mainframe.grid(column=0,row=0, sticky=(N,W,E,S) ) # Add grid to frame 
mainframe.columnconfigure(0, weight = 1)
mainframe.rowconfigure(0, weight = 1)
mainframe.pack(pady = 100, padx = 100)

# Create a Tkinter variable
tkvar = StringVar(root)

# Dictionary with options
choices = genre_list
tkvar.set("action") # Set the default option

popupMenu = OptionMenu(mainframe, tkvar, *choices)
Label(mainframe, text="What is your favorite genre?").grid(row = 1, column = 1)
popupMenu.grid(row = 2, column =1)

# on change dropdown value
def change_dropdown(*args):
    print( tkvar.get() )

# link function to change dropdown
tkvar.trace('w', change_dropdown)

root.mainloop()

KeyboardInterrupt: 

In [ ]:
from tkinter import *

OPTIONS = ["Drama","Action","Romance"] #Replace with genre list 

master = Tk()

variable = StringVar(master)
variable.set(OPTIONS[0]) # default value

w = OptionMenu(master, variable, *OPTIONS)
w.pack()

def ok():
    print ("value is:" + variable.get())

button = Button(master, text="OK", command=ok)
button.pack()

mainloop()